# Using AQS API for Historical AQI

In [27]:
import configparser
import json
import pandas as pd
import requests
import urllib3
from pathlib import Path
from urllib3.util.ssl_ import create_urllib3_context

In [3]:
config = configparser.ConfigParser()
config.read(Path.home() / '.config' / 'config.ini')

['/home/pybokeh/.config/config.ini']

In [4]:
aqs_api_key = config['aqs_api']['key']
email = config['aqs_api']['email']

In [9]:
aqs_api_url = f"https://aqs.epa.gov/data/api/list/states?email={email}&key={aqs_api_key}"

Some US government sites use legacy SSL verification, so need to create an SSL "context" that will adjust to using the legacy version
- [GH issue](https://github.com/urllib3/urllib3/issues/2653)
- [Stackoverflow](https://stackoverflow.com/questions/71603314/ssl-error-unsafe-legacy-renegotiation-disabled/71646353#71646353)

#### Obtaining state fips code

In [14]:
ctx = create_urllib3_context()
ctx.load_default_certs()
ctx.options |= 0x4  # ssl.OP_LEGACY_SERVER_CONNECT

with urllib3.PoolManager(ssl_context=ctx) as http:
    r = http.request("GET", aqs_api_url)
    print(r.status)

200


In [16]:
type(r)

urllib3.response.HTTPResponse

In [24]:
data = json.loads(r.data.decode('utf-8'))

In [25]:
type(data)

dict

In [26]:
data

{'Header': [{'status': 'Success',
   'request_time': '2023-06-07T18:41:29-04:00',
   'url': 'https://aqs.epa.gov/data/api/list/states?email=daniel_jay_kim@protonmail.com&key=bolefox48',
   'rows': 56}],
 'Data': [{'code': '01', 'value_represented': 'Alabama'},
  {'code': '02', 'value_represented': 'Alaska'},
  {'code': '04', 'value_represented': 'Arizona'},
  {'code': '05', 'value_represented': 'Arkansas'},
  {'code': '06', 'value_represented': 'California'},
  {'code': '08', 'value_represented': 'Colorado'},
  {'code': '09', 'value_represented': 'Connecticut'},
  {'code': '10', 'value_represented': 'Delaware'},
  {'code': '11', 'value_represented': 'District Of Columbia'},
  {'code': '12', 'value_represented': 'Florida'},
  {'code': '13', 'value_represented': 'Georgia'},
  {'code': '15', 'value_represented': 'Hawaii'},
  {'code': '16', 'value_represented': 'Idaho'},
  {'code': '17', 'value_represented': 'Illinois'},
  {'code': '18', 'value_represented': 'Indiana'},
  {'code': '19', 'v

In [30]:
pd.json_normalize(data['Data']).rename(columns={'code': 'fips_code', 'value_represented': 'state_name'})

,fips_code,state_name
0,01,Alabama
1,02,Alaska
2,04,Arizona
3,05,Arkansas
4,06,California
5,08,Colorado
6,09,Connecticut
7,10,Delaware
8,11,District Of Columbia
9,12,Florida


In [33]:
pd.json_normalize(data['Data']).rename(columns={'code': 'fips_code', 'value_represented': 'state_name'}).to_csv('../data/state_fips.csv', index=False, encoding='utf-8')

In [ ]:
ctx = create_urllib3_context()
ctx.load_default_certs()
ctx.options |= 0x4  # ssl.OP_LEGACY_SERVER_CONNECT

with urllib3.PoolManager(ssl_context=ctx) as http:
    r = http.request("GET", aqs_api_url)
    print(r.status)